# Exercise 2: Regression

[https://www.kaggle.com/datasets/sohier/calcofi](https://www.kaggle.com/datasets/sohier/calcofi)

Use the `bottle.csv` data from the CalCofi dataset to determine the most important factors of T_degC

1. Use the EDA you already have done in Julia
2. Use Julia to do linear regressions combining different sets of independent variables until you find the combination that gives you better predictions
3. Submit your project following the submittion Guidelines

In [ ]:
# Importar bibliotecas necesarias
using DataFrames
using XLSX
using StatsBase
using Plots
import CSV
#using StatsPlots

# Cargar el archivo
#file_path = "bottle_sample.xlsx"
file_path = "DATA/Calcofi/bottle.csv"
#data = XLSX.readtable(file_path, "Hoja1")  # Cambiar "Hoja1" al nombre real de la hoja si es necesario
df = DataFrame(CSV.File(file_path; normalizenames=true, stringtype=String))


# Convertir a DataFrame
df = DataFrame(df)
println("Datos cargados correctamente.")

Datos cargados correctamente.


In [ ]:
using DataFrames
using CSV
using GLM
using StatsBase
using Combinatorics

# Paso 1: Cargar el archivo de datos con opciones para manejar caracteres problemáticos
#file_path = "bottle_sample.xlsx"
file_path = "DATA/Calcofi/bottle.csv"

# Intentar normalizar nombres y manejar datos con codificación problemática
try
    #df = DataFrame(XLSX.readtable(file_path, "Hoja1"))  # Cambiar "Hoja1" al nombre real de la hoja si es necesario
    df = DataFrame(CSV.File(file_path; normalizenames=true, stringtype=String))
    catch e
    println("Error al cargar el archivo: $(e)")
    println("Intenta convertir el archivo a un formato CSV limpio o verifica la codificación.")
    return
end

# Limpiar nombres de columnas
clean_names = [replace(col, r"[^\w]+" => "_") for col in names(df)]
rename!(df, Dict(names(df) .=> clean_names))

println("Nombres de las columnas después de limpiar:")
println(names(df))

# Paso 2: Limpieza y preparación de datos
for col in names(df)
    df[!, col] = [
        isa(x, AbstractString) ? try parse(Float64, x) catch missing end :
        (isa(x, Number) ? Float64(x) : missing)
        for x in df[!, col]
    ]
end

# Asegurarte de que la columna `T_degC` existe antes de continuar
if "T_degC" ∉ names(df)
    println("Error: La columna 'T_degC' no existe en el archivo después de la limpieza.")
    return
end

Nombres de las columnas después de limpiar:
["Cst_Cnt", "Btl_Cnt", "Sta_ID", "Depth_ID", "Depthm", "T_degC", "Salnty", "O2ml_L", "STheta", "O2Sat", "Oxy_µmol_Kg", "BtlNum", "RecInd", "T_prec", "T_qual", "S_prec", "S_qual", "P_qual", "O_qual", "SThtaq", "O2Satq", "ChlorA", "Chlqua", "Phaeop", "Phaqua", "PO4uM", "PO4q", "SiO3uM", "SiO3qu", "NO2uM", "NO2q", "NO3uM", "NO3q", "NH3uM", "NH3q", "C14As1", "C14A1p", "C14A1q", "C14As2", "C14A2p", "C14A2q", "DarkAs", "DarkAp", "DarkAq", "MeanAs", "MeanAp", "MeanAq", "IncTim", "LightP", "R_Depth", "R_TEMP", "R_POTEMP", "R_SALINITY", "R_SIGMA", "R_SVA", "R_DYNHT", "R_O2", "R_O2Sat", "R_SIO3", "R_PO4", "R_NO3", "R_NO2", "R_NH4", "R_CHLA", "R_PHAEO", "R_PRES", "R_SAMP", "DIC1", "DIC2", "TA1", "TA2", "pH2", "pH1", "DIC_Quality_Comment"]


In [ ]:
"T_degC" ∈  names(df)

true

In [ ]:
df = dropmissing(df, [:T_degC])

# Paso 3: Inspeccionar la correlación para identificar posibles variables relevantes
numeric_cols = filter(col -> eltype(df[!, col]) <: Number && !all(ismissing.(df[!, col])), names(df))
numeric_df = select(df, numeric_cols)

# Calcular la matriz de correlación
correlation_matrix = cor(Matrix(numeric_df), dims=1)
println("Matriz de correlación:")
println(correlation_matrix)

# Paso 4: Selección inicial de variables independientes
# Identificar las variables con mayor correlación con T_degC
cor_with_target = [cor(numeric_df[:, col], numeric_df[:, "T_degC"]) for col in numeric_cols if col != "T_degC"]
sorted_indices = sortperm(abs.(cor_with_target), rev=true)
selected_vars = numeric_cols[sorted_indices[1:5]] # Seleccionar las 5 más correlacionadas
println("Variables seleccionadas:")
println(selected_vars)

# Paso 5: Crear modelos de regresión
best_model = nothing
best_r2 = -Inf

for subset_size in 1:length(selected_vars)
    for subset in combinations(selected_vars, subset_size)
        try
            # Crear fórmula dinámica
            terms = join(subset, " + ")
            formula = @eval @formula(T_degC ~ $(Meta.parse(terms)))

            # Ajustar el modelo lineal
            model = lm(formula, df)

            # Calcular R² manualmente
            y_true = df[:, :T_degC]
            y_pred = predict(model, df)
            ss_total = sum((y_true .- mean(y_true)).^2)
            ss_residual = sum((y_true .- y_pred).^2)
            r2 = 1 - (ss_residual / ss_total)

            # Comparar y actualizar el mejor modelo
            if r2 > best_r2
                best_model = model
                best_r2 = r2
            end
        catch e
            println("Error al ajustar modelo con variables: $subset. Error: $(e)")
        end
    end
end

println("Mejor modelo encontrado:")
println(best_model)
println("R² del mejor modelo: $best_r2")

Matriz de correlación:
[1.0 0.9993439435591516 -0.16565094745771503 0.0893291280990099 -0.10452452232921687 -0.16564826321900222 -0.16537795168162533; 0.9993439435591516 1.0 -0.16509880297936272 0.09011817438960189 -0.10520676211184322 -0.16509604413407358 -0.1648182246037856; -0.16565094745771503 -0.16509880297936272 1.0 -0.6812014451784331 0.08143846416836109 0.9999999972609181 0.9999939131726389; 0.0893291280990099 0.09011817438960189 -0.6812014451784331 1.0 -0.0665066519051583 -0.681199943358432 -0.6798516522321398; -0.10452452232921687 -0.10520676211184322 0.08143846416836109 -0.0665066519051583 1.0 0.08143972809859315 0.08130542469424724; -0.16564826321900222 -0.16509604413407358 0.9999999972609181 -0.681199943358432 0.08143972809859315 1.0 0.9999939206455185; -0.16537795168162533 -0.1648182246037856 0.9999939131726389 -0.6798516522321398 0.08130542469424724 0.9999939206455185 1.0]
Variables seleccionadas:
["Depthm", "RecInd", "R_Depth", "Btl_Cnt", "Cst_Cnt"]
Mejor modelo encontr